In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import os

# 1. 파일 경로 설정
DATA_PATH = '../../data/raw/'
NPY_PATH = './oof_data/'
npy_m1 = 'exp11_sota_AUC_0.77212.npy'
npy_m2 = 'exp12_auc_AUC_0.77233.npy'
npy_m7 = 'exp25_m7_Refined_AUC_0.77375.npy'

# 2. 데이터 로드 및 동기화 (필수)
train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
mask_family = train['familysize'] <= 50
a_cols = [col for col in train.columns if col.endswith('A') and col.startswith('Q')]
mask_std_global = train[a_cols].std(axis=1) > 0

# 데이터 로드
p1 = np.load(os.path.join(NPY_PATH, npy_m1)).ravel()
p2 = np.load(os.path.join(NPY_PATH, npy_m2)).ravel()
p7 = np.load(os.path.join(NPY_PATH, npy_m7)).ravel()

# 동기화 Slicing
df_family_filtered = train[mask_family]
mask_final_sync = df_family_filtered[a_cols].std(axis=1) > 0

p1 = p1[mask_final_sync.values]
p2 = p2[mask_final_sync.values]
p7 = p7 # m7은 원본 사용
y_true = train[mask_family][mask_final_sync]['voted'].values

# 방향성 보정
def check_flip(pred, true):
    score = roc_auc_score(true, pred)
    return (1 - pred) if score < 0.5 else pred

p1 = check_flip(p1, y_true)
p2 = check_flip(p2, y_true)
p7 = check_flip(p7, y_true)

# 랭크 변환
r1 = pd.Series(p1).rank(pct=True)
r2 = pd.Series(p2).rank(pct=True)
r7 = pd.Series(p7).rank(pct=True)

# 3. 앙상블 방식 비교 (가중치는 최적 비율 33:33:34 고정)
w1, w2, w7 = 0.33, 0.33, 0.34

# Method 1: 산술 평균 (Arithmetic Mean - 기존 방식)
score_arith = (r1 * w1) + (r2 * w2) + (r7 * w7)
auc_arith = roc_auc_score(y_true, score_arith)

# Method 2: 기하 평균 (Geometric Mean - 의견 일치 중시)
# (r1^w1 * r2^w2 * r7^w7)
score_geo = (r1 ** w1) * (r2 ** w2) * (r7 ** w7)
auc_geo = roc_auc_score(y_true, score_geo)

# Method 3: Power 평균 (Power Mean p=2 - 강한 확신 중시)
# ((w1*r1^2 + w2*r2^2 + w3*r7^2))
score_power = (w1 * (r1**2)) + (w2 * (r2**2)) + (w7 * (r7**2))
auc_power = roc_auc_score(y_true, score_power)

print("="*40)
print(f"1. 산술 평균 (Arithmetic): {auc_arith:.6f}")
print(f"2. 기하 평균 (Geometric) : {auc_geo:.6f}")
print(f"3. 제곱 평균 (Power Mean): {auc_power:.6f}")
print("="*40)

# 승자 추천
best_score = max(auc_arith, auc_geo, auc_power)
if best_score == auc_arith: print("추천: 기존 산술 평균 (Arithmetic)")
elif best_score == auc_geo: print("추천: 기하 평균 (Geometric) - 의견 일치 강화")
else: print("추천: 제곱 평균 (Power Mean) - 상위권 변별력 강화")

1. 산술 평균 (Arithmetic): 0.773604
2. 기하 평균 (Geometric) : 0.773635
3. 제곱 평균 (Power Mean): 0.773563
추천: 기하 평균 (Geometric) - 의견 일치 강화


In [2]:
import pandas as pd
import numpy as np
import os

# ==========================================
# [설정] 최종 승자: 기하 평균 (Geometric)
# ==========================================
METHOD = 'geometric'

DATA_PATH = '../../data/raw/'
SUB_PATH = './submissions/'
sample_sub = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))

# 1. 모델 데이터 로드
def load_and_align(filename):
    df = pd.read_csv(os.path.join(SUB_PATH, filename))
    if 'index' in df.columns: df = df.set_index('index').reindex(sample_sub['index']).reset_index()
    elif 'id' in df.columns: df = df.set_index('id').reindex(sample_sub['id']).reset_index()
    return df['voted'].values

print(">>> 데이터 로딩 중...")
s1 = load_and_align('0130-1923.csv')               # m1 (Base)
s2 = load_and_align('Pure_0781_AUC_0.77322.csv')   # m2 (Private Ace)
s7 = load_and_align('m7_Refined_test_preds.csv')   # m7 (Deep FE)

# 2. 방향성 자동 보정 (필수)
if np.corrcoef(s1, s7)[0, 1] < 0:
    print(">>> m7 역방향 감지 -> 자동 반전 적용")
    r7_raw = 1.0 - pd.Series(s7).rank(pct=True)
else:
    r7_raw = pd.Series(s7).rank(pct=True)

# 랭크 변환
r1 = pd.Series(s1).rank(pct=True)
r2 = pd.Series(s2).rank(pct=True)
r7 = r7_raw

# 3. Geometric Mean Ensemble (33:33:34)
# 로컬 테스트 최고점 조합
w1, w2, w7 = 0.33, 0.33, 0.34

print(f">>> [GEOMETRIC MEAN] 방식으로 앙상블 생성 중...")
# 기하 평균 공식: (r1^w1 * r2^w2 * r7^w7)
final_score = (r1 ** w1) * (r2 ** w2) * (r7 ** w7)

# 4. Scale Restoration (m1 분포 주입 - 가장 안정적)
m1_sorted = np.sort(s1)
indices = final_score.rank(method='first').astype(int).values - 1
final_voted = m1_sorted[indices]

# 5. 저장
submission = sample_sub.copy()
submission['voted'] = final_voted
output_name = os.path.join(SUB_PATH, '48_FINAL_GEOMETRIC_SHOT.csv')
submission.to_csv(output_name, index=False)

print("-" * 30)
print(f"파일 생성 완료: {output_name}")
print("전략: 기하 평균 (Geometric Mean)")
print("근거: 로컬 시뮬레이션 1위 (0.773635)")
print("목표: Noise Filtering 효과를 통한 0.7817+ 돌파")
print("-" * 30)

>>> 데이터 로딩 중...
>>> m7 역방향 감지 -> 자동 반전 적용
>>> [GEOMETRIC MEAN] 방식으로 앙상블 생성 중...
------------------------------
파일 생성 완료: ./submissions/48_FINAL_GEOMETRIC_SHOT.csv
전략: 기하 평균 (Geometric Mean)
근거: 로컬 시뮬레이션 1위 (0.773635)
목표: Noise Filtering 효과를 통한 0.7817+ 돌파
------------------------------
